In [1]:
import numpy as np

In [2]:
class PSU:
    def __init__(self, Vout, LS_Ron, Iout, Vin):
        self.Vout = Vout
        self.LS_Ron = LS_Ron
        self.Iout = Iout
        self.Vin = Vin
        self.HS_Ron = LS_Ron
    
    def duty_cycle(self):
        D = (self.Vout + (self.LS_Ron * self.Iout))/(self.Vin - (self.HS_Ron * self.Iout) + (self.LS_Ron * self.Iout))
        return D
    
    def HS_loss(self):
        D = self.duty_cycle()
        P_HS_cond = D * self.HS_Ron * self.Iout**2 #Watts
        return P_HS_cond



In [3]:
model = PSU(np.array([3.2, 3.3, 3.4]), 
            np.array([6.3, 8.3, 11])*1e-3, 
            np.array([2.64, 3.03, 3.45]), 
            np.array([13.3, 13.35, 13.37]))

In [45]:
model.duty_cycle()

array([0.24185203, 0.24907483, 0.25713912])

In [4]:
model.HS_loss()

array([0.01061936, 0.01897987, 0.03366658])

In [2]:
# Duty Cycle: Expected Result for VBat_range = 13.5V -> D3v3 = [0.242, 0.249, 0.257]. OK!

# Variables Definition
Vout =  np.array([3.2, 3.3, 3.4])       #Vpre_static
LS_Ron =  np.array([6.3, 8.3, 11])*1e-3 #Rds_on1 
Iout = np.array([2.64, 3.03, 3.45])     #Iload_3V3M 
Vin = np.array([13.3, 13.35, 13.37])    #Vpre_in V -> INPUT!!!!
HS_Ron = LS_Ron                         #Rds_on1 

# Formula
D = (Vout + (LS_Ron * Iout))/(Vin - (HS_Ron * Iout) + (LS_Ron * Iout))
D   

array([0.24185203, 0.24907483, 0.25713912])

In [39]:
# Conduction loss in the High/Low-side MOSFET


# High-Side: Expected value ->[10.6, 18.9, 33.7] mW
# P_FET_3V3M_HS_cond_loss
P_HS_cond = D * HS_Ron * Iout**2 #Watts
P_HS_cond

array([0.01061936, 0.01897987, 0.03366658])

In [42]:
model.HS_loss(D)

array([0.01061936, 0.01897987, 0.03366658])

In [ ]:
# Low-Side: Expected value ->[33.2, 57.1, 97.3] mW
# P_FET_3V3M_LS_cond_loss
P_LS_cond = (1-D) * LS_Ron * Iout**2 #Watts

In [4]:
#HS Switching-loss in the MOSFET
#PFET_3V3M_SW_HS_loss: Expected result -> [77.9, 94.9, 114.4]mW

#Variables definition
HS_Tsw = 5.17*1e-9                  #TSW_vpre [ns]
Fsw = np.array([430, 455, 489])*1e3 #fSW_pre [kHz]

#Formula
P_HS_sw = (Vin * Iout) * HS_Tsw * Fsw #Watts
P_HS_sw*1e3

array([ 78.0574872 ,  95.15373367, 116.61378845])

In [5]:
#LS Body diode Reverse recovery-loss
#PFET_3V3M_SW_LS_loss: Expected result -> [3.808, 5.712, 10.267]

#Variables definition
Vbody_diode = np.array([0.65, 0.8, 1.2]) #VForward_Voltage1
LS_Tsw = HS_Tsw                          #TSW_vpre [ns]

#Formula
P_LS_sw = (Vbody_diode * Iout) * LS_Tsw * Fsw
P_LS_sw*1e3

array([ 3.8148396,  5.7020964, 10.4664582])

In [6]:
#LS Body diode Reverse recovery-loss
#PFET_3V3M_RR_LS_loss: Expected value -> [343.1, 364.3, 385] mW

#Variables definition
LS_QRR = 120*1e-9 #QRR_charge1 [nC]

#Formula
P_Qrr = LS_QRR * Vin * Fsw / 2 #[W]
P_Qrr*1e3

array([343.14  , 364.455 , 392.2758])

In [7]:
#LS Dead time-loss in the MOSFET body diode
#PFET_3V3M_DT_LS_loss: Expected value -> [29.48, 44.22, 79.49]mW

#Variables definition
DTr = 20*1e-9 #DTrising [ns]
DTf = 20*1e-9 #DTfalling [ns]

#Formula
P_DT = Vbody_diode * Iout * (DTr + DTf) * Fsw #Watts
P_DT*1e3

array([29.5152, 44.1168, 80.9784])

In [8]:
#Inductor Ripple current
#AI_L_3V3M: Expected result -> [1.05, 1.22, 1.48]

#Variables Definition
L = np.array([5.4, 4.5, 3.6])*1e-6 #LVPRE [microH] -> For some reason this one is reversed???????

#Formula
AIL = (Vin - Vout - HS_Ron * Iout) * D / (Fsw * L) # Amperes
AIL

array([1.05025109, 1.21950577, 1.45076038])

In [9]:
#Power Dissipated in the inductor
#P_Dissip_L_3V3M: Expected result -> [88.83, 176.33, 329.08] mWatts

#Variables definition
DCR = np.array([12.61, 19, 27.23])*1e-3 #R_DC_L_VPRE_temp [mOhms]

#Formula
P_L = DCR * (Iout**2 + AIL**2/12) #Watts
P_L*1e3

array([ 89.0457539 , 176.79182436, 328.88100297])

In [10]:
#Peak inductor current
#I_L_3V3M_PEAK: Expected result -> [3.16, 3.64, 4.19] A

Ipeak = Iout + AIL / 2
Ipeak

array([3.16512554, 3.63975289, 4.17538019])

In [11]:
#Input capacitors Power Dissipated
#P_Dissip_Cin_3V3M: Expected result -> [5.39, 7.24, 9.61] mWatts

#Variables definition
ESR_Cin = 4.23*1e-3 #ESR_3V3M_IN [mOhms]

#Formula
I_Cin_rms = Iout * (D * ((1-D) + ((1-D) * HS_Tsw * Vout /(Iout * L))/12))**0.5
P_Cin = ESR_Cin * I_Cin_rms**2
P_Cin

array([0.00540622, 0.00726436, 0.00961846])

In [12]:
#P_Dissip_3V3M_total Matrix
P_Dissip_Total_mat = P_HS_cond + P_LS_cond + P_HS_sw + P_L + P_Qrr + P_Cin
P_Dissip_Total_mat

array([0.55955794, 0.71986639, 0.97831655])

In [13]:
#P_Dissip_3V3M_Total_Vector

P_IC = np.array([83.93, 88.81, 93.69])*1e-3 # PMIC IC power dissipated [mW] P_Dissip_IC_3V3M ??? This comes from previous formula????

P_Dissip_Total_vec = P_LS_sw + P_DT + P_IC
P_Dissip_Total_vec

array([0.11726004, 0.1386289 , 0.18513486])

In [14]:
#P_Dissip_3V3M_Total: Expected value -> [0.676, 0.858, 1.152] W

Pdis = P_Dissip_Total_mat + P_Dissip_Total_vec
Pdis

array([0.67681798, 0.85849529, 1.16345141])

In [15]:
#P out

Pout = Iout * Vout #P3v3M [Watts]
Pout

array([ 8.448,  9.999, 11.73 ])

In [22]:
# Efficiency n (nu): Expected value -> [0.911, 0.921, 0.926]

n = np.flip(Pout) / (np.flip(Pout) + np.flip(Pdis))
n.round(3)

array([0.91 , 0.921, 0.926])

In [23]:
# Input battery or whatever
# I_IN_3V3M
I_in = Iout * Vout / (n * Vin)
I_in.round(3)

array([0.698, 0.813, 0.948])

In [24]:
n_test = np.array([0.911, 0.921, 0.926])
I_in = Iout * Vout / (n_test * Vin)
I_in.round(3)

array([0.697, 0.813, 0.947])

In [26]:
P_in = I_in * Vin
P_in

array([ 9.27332602, 10.85667752, 12.66738661])